In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 있는 신문사 - 연합뉴스, 이데일리)

import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
def naver_news_crawling(query, start_date, end_date, s_from, e_to, page):
    title_list=[]
    date_list=[]
    contents_list=[]
    #최대5페이지까지
    while page != 0:
        url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds="+start_date+"&de="+ end_date +"&cluster_rank=10&mynews=1&office_type=2&office_section_code=8&news_office_checked=1001&nso=so:r,p:from"+s_from+"to"+e_to+",a:all&start="+str(page)+"&refresh_start=0"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        req = requests.get(url,headers=header)
        cont = req.text
        soup = BeautifulSoup(cont, 'html.parser')
        for urls in soup.select("a.info"):
            try :
                if urls["href"].startswith("https://news.naver.com"):
                    title_text, date_text, contents_text= get_news(urls["href"])  
                    title_list.append(title_text)
                    date_list.append(date_text)
                    contents_list.append(contents_text)
            except Exception as e:
                print(e) 
                continue

In [ ]:
news_company = {'연합뉴스':'1001','이데일리':'2227'}
news_company['연합뉴스']

'1001'

In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 있는 신문사 - 연합뉴스, 이데일리)

import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

def get_news(url):      
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110                                   Safari/537.36'}
    html = requests.get(url, headers=header)
    bsoup = BeautifulSoup(html.content, 'html.parser')    
    # 기사 제목
    title = bsoup.select('h3#articleTitle')[0].text
    # 날짜 
    pdate = bsoup.select('.t11')[0].get_text()[:10]
    # 기사 본문
    content = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    ctext = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
    return title, pdate, ctext

def naver_news_crawling(query, start_date, end_date, s_from, e_to, page):
    
    title_list=[]
    date_list=[]
    contents_list=[]
    
    # 언론사에 따라 url주소의 언론사 숫자 변경
    news_company = {'연합뉴스':'1001','이데일리':'2227'}
    while page != 0:
  
        url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds="+start_date
        +"&de="+ end_date +"&cluster_rank=10&mynews=1&office_type=2&office_section_code=8&"
        +"news_office_checked="+news_company['연합뉴스']+"&nso=so:r,p:from"+s_from+"to"+e_to+",a:all&start="+str(page)+"&refresh_start=0"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110                                   Safari/537.36'}
        req = requests.get(url,headers=header)
        cont = req.text
        soup = BeautifulSoup(cont, 'html.parser')

        for urls in soup.select("a.info"):
            try :


                if urls["href"].startswith("https://news.naver.com"):
                   
                    title_text, date_text, contents_text= get_news(urls["href"])
                    
                    title_list.append(title_text)
                    date_list.append(date_text)
                    contents_list.append(contents_text)
                    
            except Exception as e:
                print(e) 
                continue

        # 네이버 검색결과의 다음 페이지 버튼을 가져온다.
        btn_next = soup.select_one('#main_pack > div.api_sc_page_wrap > div > a.btn_next')
        # 다음 페이지 버튼이 없으면(검색 결과 없을 경우) 크롤링 하지 않고 다음 날짜 링크로 가기
        if btn_next == None:
            break
        
        # 페이지의 다음 버튼이 비활성화 될때(마지막 페이지)까지 페이지 넘기기
        is_last = btn_next['aria-disabled']
        # 마지막 페이지일 경우 다음 날짜로 가기 위해 while 종료
        if is_last == "true":
            break
        # 네이버 페이지가 1, 11, 21, 31 순으로 가기 때문에 10씩 더해준다.
        else:
            page += 10

    # 데이터 프레임 형식으로 만들어 데이터 반환   
    result={}   
    result= {'date' : date_list, 'title':title_list,'contents': contents_list}
    
    df=[]
    df=pd.DataFrame(result)
    
    return df

error_date=[]

page = 1
query = "금리" 
date = datetime.date(2021,1,1) #시작날짜
cur_date=date.strftime('%Y.%m.%d')
e_to = s_from = cur_date.replace(".","")
while(cur_date!="2022.01.01"):#끝날짜 
    try:
        print(""+cur_date+"날짜의 크롤링을 시작합니다")
        new_df = naver_news_crawling(query, cur_date, cur_date, s_from, e_to, page)
        new_df=new_df.drop_duplicates()

        if len(new_df) > 0:
            new_df['title'] = new_df['title'].str.encode('utf-8', 'ignore').str.decode('utf-8')
            new_df['contents'] = new_df['contents'].str.encode('utf-8', 'ignore').str.decode('utf-8')
            new_df.to_csv(r"금리2021.csv",encoding='utf-8-sig',index=False, header=False, mode='a')
         
        date += datetime.timedelta(days=1)
        cur_date=date.strftime('%Y.%m.%d')
        e_to = s_from = cur_date.replace(".","")
        print("성공")
    except Exception as e:
        error_date.append(cur_date)
        print(e)
        print("실패")
        continue
    
print("크롤링을 종료합니다")

In [ ]:
# 1. Preparing corpus - 뉴스기사 크롤링 (네이버링크가 없는 신문사 - 연합인포맥스)
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

titles = []
dates = []
contents = []
first_page = 1
last_page = 1001

for i in tqdm(range(first_page,last_page)):
    urls = []
    url = 'https://news.einfomax.co.kr/news/articleList.html?page='+str(i)+'&total=157314&box_idxno=&sc_area=A&view_type=sm&sc_section_code=&sc_level=&sc_article_type=&sc_sdate=2008-01-01&sc_edate=2021-12-31&sc_order_by=E&sc_word=%EA%B8%88%EB%A6%AC&sc_andor=OR&sc_word2='
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news = soup.select('section[class="article-list-content type-sm text-left"] div[class="list-block"]')

    for i in range(len(news)):
        data = soup.select_one('#user-container > div.float-center.max-width-1080 > div.user-content.list-wrap > section > article > div.article-list > section > div:nth-of-type('+str(+i+1)+') > p > a')
        urls.append('https://news.einfomax.co.kr'+(data['href']))

    for href in urls:
        response = requests.get(href)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one('div[class="article-head-title"]')
        date= soup.select('div[class="info-text"] li')
        s = date[1].text
        content =  soup.select_one('#article-view-content-div')
        # 기사 날짜 정보 리스트에 append
        dates.append(s[4:15])
        # 기사 제목 정보 리스트에 append
        titles.append(title.text)
        # 기사 내용 정보 리스트에 append
        contents.append(content.text)

news_df = pd.DataFrame({'title':titles, 'date':dates,'content':contents})
news_df.to_csv(f'인포맥스{first_page}-{last_page}.csv',encoding='utf-8-sig',index=False)

In [ ]:
pip install selenium

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = "/content/drive/MyDrive/workspace/bok/발표용 코드정리/chromedriver.exe"
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    url_list = list()
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수 리스트
    periods_list = list()
    idx = 1
    while True:
        try:
            url_xpath = '//*[@id="video-title"]'
            img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
            period_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[2]'
            # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')

            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 8 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)

            # 썸네일 주소를 리스트에 저장
            image = driver.find_element_by_xpath(url_xpath)
            img_url = image.get_attribute('href')
            image_list.append(img_url)


            # 썸네일 주소를 리스트에 저장
            image = driver.find_element_by_xpath(img_xpath)
            img_url = image.get_attribute('src')
            image_list.append(img_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # 조회수를 리스트에 저장
            view = driver.find_element_by_xpath(viewcnt_xpath)
            view_list.append(view.text)

            # 시간을 리스트에 저장
            period = driver.find_element_by_xpath(period_xpath)
            periods_list.append(period)
            print(idx, title.text, view.text, period.text, img_url)

            idx += 1

        except Exception as e:
            print()
            print(e)
            break
    assert len(image_list) == len(title_list)
    # driver.close()
    return image_list, view_list, periods_list, title_list

# 자이언트 펭TV
url1 = 'https://www.youtube.com/c/%ED%95%9C%EB%AC%B8%EC%B2%A0TV1/videos'
image1, view1, period1, title1 = get_image_title(url1)

In [ ]:
!apt install chromium-chromedriver

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = "/content/drive/MyDrive/workspace/bok/발표용 코드정리/chromedriver.exe"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    driver.implicitly_wait(5) # or bigger second
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    url_list = list()
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수 리스트
    periods_list = list()
    idx = 1
    # while True:
    try:
        url_xpath = '//*[@id="video-title"]'
        img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
        title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
        viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
        period_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[2]'
        # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
        img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
        if img is None:
            print(idx, 'img is not loaded.')

        # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
        if idx % 8 == 0 :
            driver.execute_script('window.scrollBy(0, 1080);')
            time.sleep(2)
            driver.execute_script('window.scrollBy(0, 1080);')
            time.sleep(2)
            driver.execute_script('window.scrollBy(0, 1080);')
            time.sleep(2)

        # 썸네일 주소를 리스트에 저장
        image = driver.find_element_by_xpath(url_xpath)
        img_url = image.get_attribute('href')
        image_list.append(img_url)


        # 썸네일 주소를 리스트에 저장
        image = driver.find_element_by_xpath(img_xpath)
        img_url = image.get_attribute('src')
        image_list.append(img_url)

        # 타이틀을 리스트에 저장
        title = driver.find_element_by_xpath(title_xpath)
        title_list.append(title.text)

        # 조회수를 리스트에 저장
        view = driver.find_element_by_xpath(viewcnt_xpath)
        view_list.append(view.text)

        # 시간을 리스트에 저장
        period = driver.find_element_by_xpath(period_xpath)
        periods_list.append(period)
        print(idx, title.text, view.text, period.text, img_url)

        idx += 1

    except Exception as e:
        print()
        print(e)
        # break
    assert len(image_list) == len(title_list)
    # driver.close()
    return url_list

# 자이언트 펭TV
url1 = 'https://www.youtube.com/c/%ED%95%9C%EB%AC%B8%EC%B2%A0TV1/videos'
url_list1 = get_image_title(url1)

In [ ]:


from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
kword = input('검색어를 입력하세요 : ')
base_url = url + quote_plus(kword)
base_url

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get(base_url)

body = driver.find_element_by_css_selector('body')

# 페이지 다운시켜서 더 많은 이미지가 나오게 한다
for i in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

imgs = driver.find_elements_by_css_selector('img._img')

for idx, img in enumerate(imgs):
    # print(idx,img.get_attribute('src'))
    imgUrl = img.get_attribute('src')
    imgName = '/content/drive/My Drive/Colab Notebooks/crawling/'+kword + str(idx)+'.jpg'
    urllib.request.urlretrieve(imgUrl, imgName)

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = "/content/drive/MyDrive/workspace/bok/발표용 코드정리/chromedriver.exe"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    driver.implicitly_wait(5) # or bigger second
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    url_list = list()
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수 리스트
    periods_list = list()
    idx = 1
    while True:
        try:       
            url_xpath = '//*[@id="video-title"]'
            # img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            # viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
            # period_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[2]'
            # # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, url_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')

            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 24 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)

            # 썸네일 주소를 리스트에 저장
            address = driver.find_element_by_xpath(url_xpath)
            url2 = address.get_attribute('href')
            url_list.append(url2)
            print(idx, url2)

            # # 썸네일 주소를 리스트에 저장
            # image = driver.find_element_by_xpath(img_xpath)
            # img_url = image.get_attribute('src')
            # image_list.append(img_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # # 조회수를 리스트에 저장
            # view = driver.find_element_by_xpath(viewcnt_xpath)
            # view_list.append(view.text)

            # # 시간을 리스트에 저장
            # period = driver.find_element_by_xpath(period_xpath)
            # periods_list.append(period)
            # # print(idx, title.text, view.text, period.text, img_url)

            idx += 1

        except Exception as e:
            print()
            print(e)
            break
    # assert len(image_list) == len(title_list)
    # driver.close()
    return idx, title_list, url_list

# 자이언트 펭TV
url1 = 'https://www.youtube.com/c/%ED%95%9C%EB%AC%B8%EC%B2%A0TV1/videos'
idx_list1, title_list1 , url_list1 = get_image_title(url1)

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = "/content/drive/MyDrive/workspace/bok/발표용 코드정리/chromedriver.exe"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
    driver.implicitly_wait(5) # or bigger second
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    # url_list = list()
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    # title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수 리스트
    periods_list = list()
    idx = 1
    idx2 = 1
    while True:
        try:       
            url_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            # img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            # viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
            # period_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[2]'
            # # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, url_xpath)))
            if img is None:
                print(idx, 'url is not loaded.')

            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 24 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)

            
            url_list = list()
            title_list = list()
            # 동영상 주소를 리스트에 저장
            address = driver.find_element_by_xpath(url_xpath)
            url2 = address.get_attribute('href')
            if url2.startswith("https://www.youtube.com/watch?") == True:
                url_list.append(url2)
                
                title = driver.find_element_by_xpath(title_xpath)
                title_list.append(title.text)

                print(idx2, title.text, url2)
                idx2 += 1

            idx += 1

            url_df = pd.DataFrame({'title':title_list, 'url':url_list})
            url_df.drop_duplicates()
            if len(url_df) > 0:
                url_df.to_csv(r"/content/drive/MyDrive/workspace/bok/발표용 코드정리/한문철1.csv",encoding='utf-8-sig',index=False, header=False, mode='a')


        except Exception as e:
            print()
            print(e)
            break
    # assert len(image_list) == len(title_list)
    # driver.close()
    
   
    return 1

# 자이언트 펭TV
url1 = 'https://www.youtube.com/c/%ED%95%9C%EB%AC%B8%EC%B2%A0TV1/videos'
print(get_image_title(url1))



# if len(new_df) > 0:
#             new_df['title'] = new_df['title'].str.encode('utf-8', 'ignore').str.decode('utf-8')
#             new_df['contents'] = new_df['contents'].str.encode('utf-8', 'ignore').str.decode('utf-8')
#             new_df.to_csv(r"금리2021.csv",encoding='utf-8-sig',index=False, header=False, mode='a')
# url_df.to_csv('/content/drive/MyDrive/workspace/bok/발표용 코드정리', encoding='utf-8-sig',index=False)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


Streaming output truncated to the last 5000 lines.
1987 13430회. 졸음 운전 트럭 때문에 연속 사고 발생!! https://www.youtube.com/watch?v=HLVSuNpee-A
1988 13429회. 초등학교 바로 앞에서 민식이법 놀이 하는 아이들 https://www.youtube.com/watch?v=Xn2nU6CQUPc
1989 13428회. 앞차가 왔다갔다 조는 것 같아 상향등를 깜빡깜빡했더니, 갑자기 브레이크를 팍!! https://www.youtube.com/watch?v=bIEyRYc0bEc
1990 13427회. 8차선 중앙분리대 있는 곳에서 무단횡단하던 사람과 사고가 났습니다. 경찰은 저를 완전한 가해자로 놓고 조사했고 처벌할 듯 싶습니다. https://www.youtube.com/watch?v=oAGJZl9YyYQ
1991 13426회. 다쳐서 진단서도 있는데 국과수 마디모에서 상해 가능성 낮다고 나오면 경찰 조사관이 대인 피해 없이 단순 대물로 사건을 종결시킬 수 있나요?? https://www.youtube.com/watch?v=QFQDYRWdWdk
1992 13425회. 차와 배의 사고.. 하루만에 천여개의 댓글. 대부분의 댓글은 거의 블박차 100% 잘못이라고 했었습니다. 후방영상을 본 후, 여러분의 의견은 어떠십니까? https://www.youtube.com/watch?v=ahstiV0iW8U
1993 13424회. 지나가는 차에 손목을 툭 쳤는데 합의금 300만원 요구하고, 결국 입원했습니다 https://www.youtube.com/watch?v=1reul9j-spQ
1994 13423회. 갑자기 달려든 대형견과 충돌해 쇄골이 부러진 오토바이 운전자. 그런데 경찰은 오토바이가 개를 친 거라고 합니다. https://www.youtube.com/watch?v=I2CC9GkJVCA
1995 13422회. 블박차 25.5톤 덤프트럭입니다. 짐이 실려있는 상태여서 급제동하지 

In [ ]:
url_df